
# HWM: Adaptive Hammerstein-Wiener Modeling Toolkit

[![License](https://img.shields.io/badge/license-BSD--3--Clause-blue.svg)](LICENSE)  
[![PyPI version](https://img.shields.io/pypi/v/hwm.svg)](https://pypi.org/project/hwm/)  
[![Documentation Status](https://readthedocs.org/projects/hwm/badge/?version=latest)](https://hwm.readthedocs.io/en/latest/)

HWM is a **Python toolkit for adaptive dynamic system modeling**, designed to capture complex nonlinear and linear relationships in data through the Hammerstein-Wiener architecture. With a flexible, modular design, HWM integrates seamlessly with [Scikit-learn](https://scikit-learn.org/), enabling streamlined workflows for regression, classification, and time-series forecasting tasks.

---

# Adaptive Hammerstein-Wiener and LSTM Modeling on KDD Cup Dataset

This Jupyter Notebook demonstrates the use of the HWM toolkit for adaptive dynamic system modeling by applying both the Hammerstein-Wiener classifier and an LSTM neural network to the KDD Cup 1999 dataset. The goal is to classify network intrusions and evaluate the performance of intelligent models in handling complex, nonlinear relationships within the data.

The workflow includes:

1. **Data Loading and Resampling**: Loading the KDD Cup dataset and resampling to a manageable size for efficient processing.
2. **Data Preprocessing**: Scaling numerical features and encoding categorical variables to prepare the data for modeling.
3. **Model Training with Hammerstein-Wiener Classifier**: Utilizing the `HammersteinWienerClassifier` for classification tasks.
4. **Hyperparameter Tuning**: Applying `RandomizedSearchCV` to optimize model parameters.
5. **Evaluation and Visualization**: Assessing model performance using accuracy, prediction stability score (PSS), and time-weighted accuracy (TWA), along with plotting confusion matrices and ROC curves.
6. **LSTM Model Training**: Implementing an LSTM neural network to handle sequence-based data and comparing its performance with the Hammerstein-Wiener classifier.



## 📦 Importing Necessary Libraries

First, we import all the necessary libraries and modules required for data handling, preprocessing, modeling, and evaluation.


In [2]:
# Import standard libraries
import os
import random
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import randint, uniform
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    accuracy_score, auc, confusion_matrix, roc_curve, 
    ConfusionMatrixDisplay
)
from sklearn.model_selection import (
    RandomizedSearchCV, train_test_split
)
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential

# Import HWM toolkit modules
from hwm.estimators import HammersteinWienerClassifier
from hwm.metrics import prediction_stability_score, twa_score
from hwm.utils import resample_data

ModuleNotFoundError: No module named 'tensorflow'

## 🗄️ Setting the Data Path

Define the path where the KDD Cup dataset is located.

In [ ]:
# Set the data path
data_path = r'F:\repositories'

## 📁 Loading the KDD Cup 1999 Dataset

We load the KDD Cup 1999 dataset, which is commonly used for network intrusion detection tasks.


In [ ]:
# Define column names as per KDD Cup 1999 dataset
column_names = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes',
    'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins',
    'logged_in', 'num_compromised', 'root_shell', 'su_attempted',
    'num_root', 'num_file_creations', 'num_shells', 'num_access_files',
    'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count',
    'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate',
    'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate',
    'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
    'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate', 'label'
]

# Define continuous and categorical features
continuous_features = [
    'duration', 'src_bytes', 'dst_bytes', 'wrong_fragment', 'urgent', 'hot',
    'num_failed_logins', 'num_compromised', 'num_root', 'num_file_creations',
    'num_shells', 'num_access_files', 'count', 'srv_count', 'serror_rate',
    'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
    'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
    'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate', 'dst_host_srv_serror_rate',
    'dst_host_rerror_rate', 'dst_host_srv_rerror_rate'
]

categorical_features = [
    'protocol_type', 'service', 'flag', 'land', 'logged_in',
    'is_host_login', 'is_guest_login', 'root_shell', 'su_attempted',
    'num_outbound_cmds'
]

# Load the dataset
data = pd.read_csv(
    os.path.join(data_path, 'kddcup.data_10_percent_corrected'),
    names=column_names,
    header=None
)

# Display the first few rows
data.head()

## 🔄 Resampling the Dataset

To ensure efficient processing, we resample the dataset to 100,000 samples.


In [ ]:
# Resample the dataset to 100,000 samples for efficiency
data = resample_data(data, samples=100000, random_state=42)

# Display the shape after resampling
print(f"Resampled data shape: {data.shape}")

## 🛠️ Data Preprocessing

We preprocess the data by scaling numerical features and encoding categorical variables.

In [ ]:
# Encode the target variable: 0 for 'normal.', 1 for any attack
data['label'] = data['label'].apply(lambda x: 0 if x == 'normal.' else 1)

# Define the preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), continuous_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# Separate features and target
X = data.drop('label', axis=1)
y = data['label']

# Apply preprocessing
X_processed = preprocessor.fit_transform(X)

# Display the shape after preprocessing
print(f"Processed features shape: {X_processed.shape}")

## 🔀 Splitting the Data into Training and Testing Sets

We split the preprocessed data into training and testing sets, ensuring that the split is stratified based on the target variable.


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y.values, test_size=0.2, random_state=42, stratify=y
)

# Display the shapes of the splits
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

## 🔧 Defining a Custom ReLU Transformer

We define a custom transformer that applies the ReLU activation function, which will be used in the Hammerstein-Wiener classifier.


In [ ]:
class ReLUTransformer(BaseEstimator, TransformerMixin):
    """Custom transformer that applies the ReLU activation function."""
    
    def fit(self, X, y=None):
        """Fit method. Returns self."""
        return self

    def transform(self, X):
        """Apply ReLU activation function."""
        return np.maximum(0, X)

## 🏋️‍♂️ Initializing the Hammerstein-Wiener Classifier

We initialize the `HammersteinWienerClassifier` with the custom ReLU transformers and specified parameters.


In [ ]:
# Initialize the Hammerstein-Wiener Classifier
hw_model = HammersteinWienerClassifier(
    nonlinear_input_estimator=ReLUTransformer(),
    nonlinear_output_estimator=ReLUTransformer(),
    p=9,
    loss="cross_entropy",
    time_weighting="linear",
    batch_size="auto",
    optimizer='sgd',
    learning_rate=0.001,
    max_iter=173, 
    early_stopping=True,
    verbose=1, 
)

# Display the model parameters
print(hw_model)

## 🏋️‍♀️ Training the Hammerstein-Wiener Classifier

We train the Hammerstein-Wiener classifier using the training data.


In [ ]:
# Train the Hammerstein-Wiener Classifier
hw_model.fit(X_train, y_train)

## 🎛️ Hyperparameter Tuning with RandomizedSearchCV

We perform hyperparameter tuning using `RandomizedSearchCV` to find the best combination of parameters for the classifier.


In [ ]:
# Define the parameter grid for RandomizedSearchCV
param_distributions = {
    'p': randint(1, 10),  # Dependency order from 1 to 10
    'batch_size': randint(32, 128),  # Batch size between 32 and 128
    'optimizer': ['sgd', 'adam', 'adagrad'],  # Optimizers to choose from
    'learning_rate': uniform(0.0001, 0.01),  # Learning rate from 0.0001 to 0.01
    'max_iter': randint(50, 200)  # Max iterations between 50 and 200
}

# Initialize the Hammerstein-Wiener Classifier with fixed components
fixed_hw_model = HammersteinWienerClassifier(
    nonlinear_input_estimator=ReLUTransformer(),
    nonlinear_output_estimator=ReLUTransformer(),
    loss="cross_entropy",
    time_weighting="linear",
    verbose=0, 
    batch_size=200, 
    early_stopping=True, 
)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=fixed_hw_model,
    param_distributions=param_distributions,
    n_iter=20,  # Number of parameter settings sampled
    scoring='accuracy',  # Evaluation metric
    cv=3,  # 3-fold cross-validation
    verbose=0,
    random_state=42,
    n_jobs=-1  # Use all available cores
)

# Fit RandomizedSearchCV to find the best parameters
random_search.fit(X_train, y_train)

# Display the best parameters and the corresponding score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

## 📊 Evaluating the Hammerstein-Wiener Classifier

We use the best estimator from the hyperparameter tuning to make predictions and evaluate its performance.


In [ ]:
# Use the best estimator to make predictions
best_hw_model = random_search.best_estimator_
y_pred_hw = best_hw_model.predict(X_test)

# Evaluate the Hammerstein-Wiener Classifier
accuracy_hw = accuracy_score(y_test, y_pred_hw)
y_pred_proba_hw = best_hw_model.predict_proba(X_test)[:, 1]
pss_hw = prediction_stability_score(y_pred_proba_hw)
twa_hw = twa_score(y_test, y_pred_hw, alpha=0.9)

# Print evaluation metrics
print(f"Hammerstein-Wiener Classifier Accuracy: {accuracy_hw:.4f}")
print(f"Hammerstein-Wiener Classifier PSS: {pss_hw:.4f}")
print(f"Hammerstein-Wiener Classifier TWA: {twa_hw:.4f}")

## 🖼️ Plotting Results

We define a function to plot the Confusion Matrix and ROC Curve for the models.


In [ ]:
def plot_results(y_true, y_pred, y_pred_proba, title):
    """
    Plots the Confusion Matrix and ROC Curve for the given predictions.
    
    Parameters
    ----------
    y_true : array-like
        True target values.
    y_pred : array-like
        Predicted target values.
    y_pred_proba : array-like
        Predicted probabilities for the positive class.
    title : str
        Title for the plots.
    """
    # Confusion Matrix
    ConfusionMatrixDisplay.from_predictions(y_true, y_pred)
    plt.title(f'Confusion Matrix - {title}')
    plt.show()

    # ROC Curve
    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    roc_auc_score = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, label=f'{title} (AUC = {roc_auc_score:.4f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title('ROC Curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.show()


## 📈 Plotting Hammerstein-Wiener Classifier Results

We visualize the performance of the Hammerstein-Wiener classifier.


In [ ]:
# Plot results for Hammerstein-Wiener Classifier
plot_results(y_test, y_pred_hw, y_pred_proba_hw, 'Hammerstein-Wiener Classifier')

## 🧠 Defining and Training the LSTM Model

We implement an LSTM neural network to handle sequence-based data and compare its performance with the Hammerstein-Wiener classifier.


In [ ]:
# Determine the number of features
n_features = X_processed.shape[1]

# Define the number of timesteps
timesteps = 9  # Should match the 'p' parameter used in Hammerstein-Wiener model

def create_sequences(X, y, timesteps):
    """
    Creates input sequences and corresponding targets for LSTM.

    Parameters
    ----------
    X : ndarray
        Feature matrix.
    y : ndarray
        Target vector.
    timesteps : int
        Number of timesteps for each input sequence.

    Returns
    -------
    X_seq : ndarray
        Array of input sequences.
    y_seq : ndarray
        Array of target values corresponding to each sequence.
    """
    X_seq, y_seq = [], []
    for i in range(len(X) - timesteps):
        X_seq.append(X[i:i + timesteps])
        y_seq.append(y[i + timesteps])
    return np.array(X_seq), np.array(y_seq)

# Create sequences for LSTM
X_train_seq, y_train_seq = create_sequences(X_train, y_train, timesteps)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, timesteps)

# Verify the shapes
print(f'X_train_seq shape: {X_train_seq.shape}')
print(f'y_train_seq shape: {y_train_seq.shape}')
print(f'X_test_seq shape: {X_test_seq.shape}')
print(f'y_test_seq shape: {y_test_seq.shape}')

## 🏗️ Building the LSTM Model

We build and compile the LSTM model using TensorFlow's Keras API.


In [ ]:
# Build the LSTM model
lstm_model = Sequential([
    LSTM(64, input_shape=(timesteps, n_features)),
    Dense(1, activation='sigmoid')
])

# Compile the model
lstm_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Define early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Display the model architecture
lstm_model.summary()

## 🏋️‍♂️ Training the LSTM Model

We train the LSTM model using the training sequences.


In [ ]:
# Train the LSTM model
lstm_history = lstm_model.fit(
    X_train_seq, y_train_seq,
    epochs=10,
    batch_size=64,
    validation_split=0.1,
    callbacks=[early_stopping],
    verbose=1
)


## 📊 Evaluating the LSTM Model

We evaluate the trained LSTM model on the test sequences and compute relevant metrics.


In [ ]:
# Evaluate the LSTM Model
lstm_loss, lstm_accuracy = lstm_model.evaluate(X_test_seq, y_test_seq, verbose=0)
y_pred_proba_lstm = lstm_model.predict(X_test_seq).flatten()
y_pred_lstm = (y_pred_proba_lstm >= 0.5).astype(int)
pss_lstm = prediction_stability_score(y_pred_proba_lstm)
twa_lstm = twa_score(y_test_seq, y_pred_lstm, alpha=0.9)

# Print evaluation metrics
print(f"LSTM Accuracy: {lstm_accuracy:.4f}")
print(f"LSTM PSS: {pss_lstm:.4f}")
print(f"LSTM TWA: {twa_lstm:.4f}")

## 📈 Plotting LSTM Model Results

We visualize the performance of the LSTM model.

In [ ]:
# Plot results for LSTM Model
plot_results(y_test_seq, y_pred_lstm, y_pred_proba_lstm, 'LSTM Model')

## 🆚 Comparing ROC Curves Between Models

We compare the ROC curves of both the Hammerstein-Wiener classifier and the LSTM model.


In [ ]:
# Compute ROC curve for Hammerstein-Wiener Classifier
fpr_hw, tpr_hw, _ = roc_curve(y_test, y_pred_proba_hw)
roc_auc_hw = auc(fpr_hw, tpr_hw)

# Compute ROC curve for LSTM Model
fpr_lstm, tpr_lstm, _ = roc_curve(y_test_seq, y_pred_proba_lstm)
roc_auc_lstm = auc(fpr_lstm, tpr_lstm)

# Plot both ROC curves
plt.figure()
plt.plot(fpr_hw, tpr_hw, label=f'Hammerstein-Wiener (AUC = {roc_auc_hw:.4f})')
plt.plot(fpr_lstm, tpr_lstm, label=f'LSTM (AUC = {roc_auc_lstm:.4f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.title('ROC Curve Comparison')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.show()

## 📝 Summary of Results

We summarize the performance metrics of both models for easy comparison.


In [ ]:
# Print Summary of Results
print("Summary of Results:")
print(f"Hammerstein-Wiener Classifier Accuracy: {accuracy_hw:.4f}")
print(f"Hammerstein-Wiener Classifier PSS: {pss_hw:.4f}")
print(f"Hammerstein-Wiener Classifier TWA: {twa_hw:.4f}")
print(f"LSTM Accuracy: {lstm_accuracy:.4f}")
print(f"LSTM PSS: {pss_lstm:.4f}")
print(f"LSTM TWA: {twa_lstm:.4f}")